**This notebook analyse data from a range of static and dynamic predictors to develop a spatio-temporal analysis of use of outside natural spaces. Using a range of static and dynamic features, a machine learning model is trained to estimate (predict) the number of visitors in outside natural spaces based on people monitoring sites at specific locations**

. People counter data for specific locations (Natural England, North Downs Way, River and Canal Trust)

. Strava Metro (pedestrian) activity data

. Socio-economic and demographic features 

. Open Street Map features

. Rural Urban classification features 

. Living England Habitat Map features

. Accessible Green and Blue infrastructure features

. Historical weather features 

. People and Natural Survey features 



1. <a href='#dest_1'>Import packages</a>

2. <a href='#dest_2'>Config file</a>

3. <a href='#dest_3'>Utility functions</a>

4. <a href='#dest_4'>Load data</a>

5. <a href='#dest_5'>Univariate model: Counter data vs Strava data</a>

6. <a href='#dest_6'>Static and dynamic features merging</a>

7. <a href='#dest_7'>Multivariate Regression model</a> 


https://github.com/pycaret/pycaret/blob/master/examples/Pycaret_2.1_Regression_EmployeePerformance.ipynb

In [ ]:
os.chdir('..')

**Import Packages**
<a id='dest_1'></a>

In [ ]:
%reload_ext autoreload
%autoreload 2

from model_packages import *
from model_config import *
from model_utils import *

# LOAD  DATASETS

<a id='dest_4'></a>

## 1. Read Census 2011 data- Socio-economic and demographic features

This is an Open dataset from https://www.nomisweb.co.uk

The dataset has been prepared in a separate notebook (Census features)

In [ ]:
df_demg=pd.read_pickle(census_locn_file)
# Area of each buffer region around the counter sites.
# Buffer zone is 5km radius around each site:pi*r^2: 78.5 sq km
area_sites_oa=df_demg.groupby('counter')['area_sq_km'].sum().reset_index()


df_demg['Density (number of persons per sq_km)']=100*df_demg['Density (number of persons per hectare)']

## 2. People monitoring counter data

**Target variable in the model (y)**

In [ ]:
# Location of the counter sites

#Natural england sites
counter_ne=assgn_lat_lon(ne_countr_locn_file,'counter')

#North Downs way sites
counter_nd=assgn_lat_lon(ndw_countr_locn_file,'counter')


# 01-2023- No training data available for these sites-
# Canal River Trust sites
counter_cr=assgn_lat_lon(crt_countr_locn_file,'Counter')\
[['Counter','latitude','longitude','geometry']]
counter_cr.rename(columns={'Counter':'counter'},inplace=True)
all_counter_location=pd.concat([counter_ne,counter_nd,counter_cr]).reset_index(drop=True)

In [ ]:
#Pre-processing of counter data- Target variable

df_count_ne=prepare_counter_data(ne_strava_data_file,2020)

In [ ]:
# North Downs Way data prep
df_count_nd=prepare_counter_data(ndw_strava_data_file,2021)

In [ ]:
#Visualisation of the time series data

df_count_line=df_count_ne.copy().sort_index().reset_index()

fig = px.line(df_count_line, x='Date', y=df_count_line.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title='People counter',markers=True)
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y")
fig.show()

In [ ]:
df_count_line_val=df_count_nd.copy().sort_index().reset_index()

fig = px.bar(df_count_line_val, x='Date', y=df_count_line_val.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title='People counter')
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y")
fig.show()

## 3. Open Street Map data

We obtain points of interest around the counter locations using openstreet map

The following takes a long time to process so we have saved the data locally for future use

In [ ]:
# check for data saved locally and create if doesnt exist, repeated for each counter data set

if os.path.isfile(data_folder+'ne_pois_df.pkl') == True:
    print("File already exists")
    ne_pois_df= pd.read_pickle(data_folder+'ne_pois_df.pkl')
else: 
    ne_pois_df=get_pois(ne_countr_locn_file,'ne')


if os.path.isfile(data_folder+'dw_pois_df.pkl') == True:
    print("File already exists")
    nd_pois_df= pd.read_pickle(data_folder+'dw_pois_df.pkl')
else: 
    nd_pois_df=get_pois(ndw_countr_locn_file,'dw')


if os.path.isfile(data_folder+'rvr_cnl_pois_df.pkl') == True:
    print("File already exists")
    cr_pois_df= pd.read_pickle(data_folder+'rvr_cnl_pois_df.pkl')
else: 
    cr_pois_df=get_pois(crt_countr_locn_file,'rvr_cnl')

In [ ]:
# All pois across all sites
df_pois_all_sites=pd.concat([ne_pois_df,nd_pois_df,cr_pois_df]).dropna(axis=1).reset_index(drop=True)

# Merge it with buffer zone areas for each site
df_pois_all_sites=df_pois_all_sites.merge(area_sites_oa.rename(columns={'counter':'site'}),on=['site'],how='inner')


num_cols=[x for x in df_pois_all_sites.columns if x not in ['area_sq_km','site']]

# Density of pois
df_pois_all_sites[num_cols]=df_pois_all_sites[num_cols].div(df_pois_all_sites['area_sq_km'],axis=0)

del df_pois_all_sites['area_sq_km']

df_pois_all_sites.to_pickle(data_folder+'pois_data_all_sites.pkl')

## Socioeconomic and Demographic Data Augmentation

Socioeconomic and demographic features taken from the census have spatial correlations and we need to infer the correlated features (using Factor Analysis)

### Define collecive features

In [ ]:
# Define collective features

df_demg['3+ people in household']=df_demg[['3 people in household','4 people in household',\
                                           '5 people in household','6 people in household',\
                                           '7 people in household','8 or more people in household']].sum(axis=1)


df_demg['1-2 people in household']=df_demg[['1 person in household','2 people in household']].sum(axis=1)



household_occupancy_ftrs=['1-2 people in household','3+ people in household']




df_demg['Age group 0-25']=df_demg[['Age 0 to 4','Age 5 to 7','Age 8 to 9', 'Age 10 to 14',\
                                   'Age 15', 'Age 16 to 17','Age 18 to 19', 'Age 20 to 24']].sum(axis=1)

df_demg['Age group 25-65']=df_demg[['Age 25 to 29', 'Age 30 to 44','Age 45 to 59', 'Age 60 to 64']].sum(axis=1)


df_demg['Age group 65+']=df_demg[['Age 65 to 74', 'Age 75 to 84','Age 85 to 89', 'Age 90 and over']].sum(axis=1)



age_ftrs=['Age group 0-25','Age group 25-65','Age group 65+']



df_demg['Household is deprived in at least 1 dimension']=df_demg[['Household is deprived in 1 dimension',\
                                                                  'Household is deprived in 2 dimensions',\
                                                                  'Household is deprived in 3 dimensions',\
                                                                  'Household is deprived in 4 dimensions']].\
sum(axis=1)



deprivation_ftrs=['Household is not deprived in any dimension','Household is deprived in at least 1 dimension']

people_density_ftrs=['Density (number of persons per sq_km)']



df_demg['Unemployed_population']=df_demg[['Unemployed: Age 16 to 24','Unemployed: Age 50 to 74',\
                                          'Unemployed: Never worked','Long-term unemployed']].sum(axis=1)

econominc_activity_ftrs=['Economically active', 'Economically Inactive','Unemployed_population']


df_demg['Population in Good Health']=df_demg[['Very good health','Good health','Fair health']].sum(axis=1)


df_demg['Population in Bad Health']=df_demg[['Bad health', 'Very bad health']].sum(axis=1)


health_ftrs=['Population in Good Health','Population in Bad Health']

df_demg['Mixed/Black/others']=df_demg[['Mixed/multiple ethnic groups',\
                                       'Black/African/Caribbean/Black British',\
                                       'Other ethnic group']].sum(axis=1)




ethnic_ftrs=['White','Asian/Asian British','Mixed/Black/others']




       
        
df_demg['2 or more cars or vans in household']=df_demg[['2 cars or vans in household',\
                                                        '3 cars or vans in household',\
                                                        '4 or more cars or vans in household']].sum(axis=1)

vehicle_ftrs=['No cars or vans in household', '1 car or van in household',\
                  '2 or more cars or vans in household']



all_ftrs_sbset=[household_occupancy_ftrs,age_ftrs,deprivation_ftrs,econominc_activity_ftrs,health_ftrs,ethnic_ftrs,\
                vehicle_ftrs]

# Get proportion of all Census features
for cols in all_ftrs_sbset:
    
    df_demg[cols]=get_proportion(df_demg,cols)
    
    

df_demg=df_demg[[item for sublist in all_ftrs_sbset for item in sublist]+\
                people_density_ftrs+['counter']+['geometry','urban_rural']+['2011 output area']].copy()

### Split the demographic data for counter sites Natural England vs North Downs Way vs River Canal Trust and compute average values of Census variables for each counter sites

In [ ]:
df_demg_ne=df_demg[df_demg['counter'].isin(list(counter_ne['counter'].values))].reset_index(drop=True)


df_demg_nd=df_demg[df_demg['counter'].isin(list(counter_nd['counter'].values))].reset_index(drop=True)


df_demg_cr=df_demg[df_demg['counter'].isin(list(counter_cr['counter'].values))].reset_index(drop=True)

### Define baseline population for regression

In [ ]:
# All Socio-economic demographic features

ftrs_to_keep_pop=[item for sublist in all_ftrs_sbset for item in sublist]+people_density_ftrs



# For regression analysis we need to define baseline population- 
# against which the regr coefs can be compared

baseline_pop=['1-2 people in household', 'Age group 65+', 'Household is not deprived in any dimension',\
'Economically Inactive' ,'Population in Good Health', 'White', '1 car or van in household'] 


ftrs_to_keep_pop=[x for x in ftrs_to_keep_pop if x not in baseline_pop]

### Compile demographic features from non-baseline population

In [ ]:
# Average proportion of Census variables for all counter sites excluding baseline population


# Natural England Counters
df_demg_ne[ftrs_to_keep_pop]=df_demg_ne[ftrs_to_keep_pop].astype(float)

df_demg_ne=df_demg_ne.groupby('counter')[ftrs_to_keep_pop].mean().reset_index()

# Merge it with buffer zone areas for each site
df_demg_ne=df_demg_ne.merge(area_sites_oa,on=['counter'],how='inner')

# normalise the census features same units as pois

num_cols=[x for x in ftrs_to_keep_pop if x not in people_density_ftrs]

#df_demg_ne[num_cols]=df_demg_ne[num_cols].div(df_demg_ne['area_sq_km'],axis=0)

del df_demg_ne['area_sq_km']


# North Downs Way Counters
df_demg_nd[ftrs_to_keep_pop]=df_demg_nd[ftrs_to_keep_pop].astype(float)

df_demg_nd=df_demg_nd.groupby('counter')[ftrs_to_keep_pop].mean().reset_index()

# Merge it with buffer zone areas for each site
df_demg_nd=df_demg_nd.merge(area_sites_oa,on=['counter'],how='inner')

# normalise the census features same units as pois

#df_demg_nd[num_cols]=df_demg_nd[num_cols].div(df_demg_nd['area_sq_km'],axis=0)

del df_demg_nd['area_sq_km']


# CRT Counters
df_demg_cr[ftrs_to_keep_pop]=df_demg_cr[ftrs_to_keep_pop].astype(float)

df_demg_cr=df_demg_cr.groupby('counter')[ftrs_to_keep_pop].mean().reset_index()

# Merge it with buffer zone areas for each site
df_demg_cr=df_demg_cr.merge(area_sites_oa,on=['counter'],how='inner')

# normalise the census features same units as pois

#df_demg_cr[num_cols]=df_demg_cr[num_cols].div(df_demg_cr['area_sq_km'],axis=0)

del df_demg_cr['area_sq_km']


all_sites_demographic=pd.concat([df_demg_ne[ftrs_to_keep_pop],df_demg_nd[ftrs_to_keep_pop],\
                                 df_demg_cr[ftrs_to_keep_pop]]).reset_index(drop=True)

### Factor Analysis on Census Features

In [ ]:
# Create factor analysis object and perform factor analysis

df_chsen=all_sites_demographic
fa = FactorAnalyzer()
fa.fit(df_chsen)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()

#Number of factors is equal to those with eigen values > 1
numb_fctr=sum(i > 1 for i in list(ev))

# re-run factor analysis with only factors whos eigenvector was > 1 in initial run 
fa = FactorAnalyzer(n_factors=numb_fctr)
fa.fit(df_chsen)


df_load=pd.DataFrame(fa.loadings_,columns=['Factor_'+str(x+1) for x in range(numb_fctr)]).\
set_index(df_chsen.columns)

print(df_load[abs(df_load)>0.5].fillna('--'))


print(fa.get_communalities())


# 1. Sum of squared loadings (variance)
# 2. Proportional variance
# 3. Cumulative variance
print(fa.get_factor_variance())

#### Inferring factor loadings

Spatial correlations amongst areas with high:
1. Proportion of deprived households, unemployed population,
   proportion of population in bad health,
2. Proportion of larger households, younger age population, Asian demographics
3. Proportion of more densely populated areas, no cars or vans, with higher proportion of population Black/Mixed ethnic background,


In [ ]:
demographic_dat_all_sites=pd.concat([df_demg_ne[ftrs_to_keep_pop+['counter']],\
                                    df_demg_nd[ftrs_to_keep_pop+['counter']],\
                                    df_demg_cr[ftrs_to_keep_pop+['counter']]]).reset_index(drop=True)


demographic_dat_all_sites.to_pickle(data_folder+'socio_economic_data_all_sites.pkl')

### Using Variance Inflation Factor to identify possible correlations between demographic and Points Of Interest features

#The Variance Inflation Factor (VIF) is a measure of colinearity 
#among predictor variables within a multiple regression. It is 
#calculated by taking the the ratio of the variance of all a given model's
#betas divide by the variane of a single beta if it were fit alone.

https://quantifyinghealth.com/vif-threshold/

In [ ]:
# merge demographic and POI features
df_pois_demo_all_sites=df_pois_all_sites.merge(demographic_dat_all_sites,left_on=['site'],\
                                               right_on=['counter'],how='inner')

df_pois_demo_all_sites=df_pois_demo_all_sites._get_numeric_data()

df_chsen=df_pois_demo_all_sites

# For each ftr, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_chsen.values, i) for \
                     i in range(df_chsen.shape[1])]
vif["features"] = df_chsen.columns
print(vif)

print('+'*100)

#select only features with VIF score <= 2.5

df_cens_remv_multi_coll=calculate_vif_(df_chsen._get_numeric_data(),\
                                       thresh=2.5)[0]

# For each ftr, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_cens_remv_multi_coll.values, i) for \
                     i in range(df_cens_remv_multi_coll.shape[1])]
vif["features"] = df_cens_remv_multi_coll.columns
print(vif)

# select only low VIF score features
low_vif_ftrs_cens=list(vif.features.values)


# Joint Census ftrs and POIS
sel_demo_pois_vrbls=low_vif_ftrs_cens

# correlation matrix of varibales with low VIF Score
get_corr_matrx(df_chsen[low_vif_ftrs_cens],0.5)

## People Counter Data

**Filter counter data for a subset of monitoring sites (where data quality is good enough based on preprocessing of counter data)**

In [ ]:
# counter sites for which we have the people counter data (after pre-processing)

# Natural England
places_ne=[x.replace("  "," ").replace(" ","_") for x in df_count_ne.columns if x not in ['Date', 'Month']]
# Only get coordinates for those sites
counter_ne=counter_ne[counter_ne['counter'].isin(places_ne)].reset_index(drop=True)
counter_ne['coordinates'] = list(zip(counter_ne.latitude, counter_ne.longitude))
site_names_ne=list(counter_ne['counter'])
# save sites with good data quality
counter_ne.to_pickle(data_folder+'counter_ne.pkl')


# North Downs Way
places_nd=[x.replace("  "," ").replace(" ","_") for x in df_count_nd.columns if x not in ['Date', 'Month']]
# Only get coordinates for those sites
counter_nd=counter_nd[counter_nd['counter'].isin(places_nd)].reset_index(drop=True)
counter_nd['coordinates'] = list(zip(counter_nd.latitude, counter_nd.longitude))
site_names_nd=list(counter_nd['counter'])
# save sites with good data quality
counter_nd.to_pickle(data_folder+'counter_nd.pkl')


# River and Canal Trust: No counter data available yet (17-02-2023)
counter_cr.to_pickle(data_folder+'counter_cr.pkl')
places_cr=df_demg_cr['counter'].unique().tolist()
site_names_cr=places_cr

## STRAVA Metro data

In [ ]:
# The following steps are required to clean up the name of folders obtained from Strava

file_names_ne=[x[0] for x in os.walk(strava_data_loc) if '_ped' in x[0]]

for x in file_names_ne:
    os.rename(x,x.replace('_1_edge_daily_2020-01-01-2022-11-30_ped',""))
    
    
file_names_nd=[x[0] for x in os.walk(val_strava_data_loc) if '_ped' in x[0]]

for x in file_names_nd:
    os.rename(x,x.replace('_1_edge_daily_2020-01-01-2022-11-30_ped',""))
    
    
file_names_nd=[x[0] for x in os.walk(canal_trust_strava_data_loc) if '_ped' in x[0]]

for x in file_names_nd:
    os.rename(x,x.replace('_1_edge_daily_2020-01-01-2022-11-30_ped',""))

In [ ]:
# Obtain Strava data

strava_count_ne=prepare_strava(site_names_ne,strava_data_loc) 

strava_count_nd=prepare_strava(site_names_nd,val_strava_data_loc)

strava_count_cr=prepare_strava(site_names_cr,canal_trust_strava_data_loc)

In [ ]:
strava_count_ne

### Processing of counter data and reshaping the data to merge with the Strava Metro data

In [ ]:
# Natural England

# data wrangling
df_count_ne_strava=df_count_ne.copy()
df_count_ne_strava.columns=[x.replace("  "," ").replace(" ","_") for x in df_count_ne_strava.columns]
df_count_ne_strava=df_count_ne_strava.stack().reset_index()
df_count_ne_strava.columns = ['Date', 'site', 'people_counter_data']
df_count_ne_strava['Date']=df_count_ne_strava['Date'].dt.to_period('M').astype("string")

print('+'*200)

# merge people counter and strava data 
df_count_ne_strava=df_count_ne_strava.merge(strava_count_ne,how='inner',left_on=['Date','site'],\
                                            right_on=['Date','site'])

In [ ]:
# North Downs Way

# data wrangling
df_count_nd_strava=df_count_nd.copy()
df_count_nd_strava.columns=[x.replace("  "," ").replace(" ","_") for x in df_count_nd_strava.columns]
df_count_nd_strava=df_count_nd_strava.stack().reset_index()
df_count_nd_strava.columns = ['Date', 'site', 'people_counter_data']
df_count_nd_strava['Date']=df_count_nd_strava['Date'].dt.to_period('M').astype("string")

print('+'*200)

# merge people counter and strava data 
df_count_nd_strava=df_count_nd_strava.merge(strava_count_nd,how='inner',left_on=['Date','site'],\
                                            right_on=['Date','site'])

In [ ]:
# Canal River Trust

df_count_cr_strava=strava_count_cr

print('+'*200)

In [ ]:
# create single data frame of strava and people counter data for all sites
strava_dat_all_sites=pd.concat([df_count_ne_strava,df_count_nd_strava, df_count_cr_strava]).reset_index(drop=True)

strava_dat_all_sites.to_pickle(data_folder+'strava_data_all_sites.pkl')

### Visualisation of correlations of counter data with Strava Metro data

In [ ]:
# Create a custom palette
cmap = sns.diverging_palette(250, 15, s=75, l=20, n=9, center="light", as_cmap=True)

# Compute corr matrix
matrix = strava_dat_all_sites.set_index('Date').corr(method="pearson")

# plot the STRAVA metro data with the highest correlation with the Counter data
matrix.sort_values(by='people_counter_data',ascending=False)['people_counter_data'][1:].plot(kind='barh',figsize=(10,7))
plt.xlabel('Pearson correlation')

# we hand-pick a feature with the highest correlation 
# with the NE counter feature: we choose total_trip_count

#### Univariate Regression Model

<a id='dest_5'></a>

In [ ]:
# Natural England: 

df_univ_ne=df_count_ne_strava[['Date','site','people_counter_data',chsen_ftr_strava]].copy()
df_univ_ne=df_univ_ne.set_index('Date').sort_index()

# Univariate regression model between people counter-Strava
df = df_univ_ne
fig = px.scatter(df, x=chsen_ftr_strava, y="people_counter_data",\
                  color="site",trendline="ols")
fig.show()

results = px.get_trendline_results(fig)

# Store univariate regression outputs
site_values=results['site'].values

str_regrn_ne=[]
for sites in site_values:
    df_site=df[df['site']==sites]
    scaler = StandardScaler()
    df_site[["people_counter_data",chsen_ftr_strava]]=scaler.fit_transform(df_site[["people_counter_data",chsen_ftr_strava]])
    lm = pg.linear_regression(df_site[['people_counter_data']],df_site[chsen_ftr_strava])
    lm['site']=sites
    str_regrn_ne.append(lm)
    
str_regrn_ne=pd.concat(str_regrn_ne).reset_index(drop=True)

str_regrn_ne[['r2','site']].drop_duplicates().set_index('site').sort_values(by='r2',ascending=False).\
plot(kind='barh',figsize=(8,6))

In [ ]:
# North Downs Way

df_univ_nd=df_count_nd_strava[['Date','site','people_counter_data',chsen_ftr_strava]].copy()
df_univ_nd=df_univ_nd.set_index('Date').sort_index()

# Univariate regression model between people counter-Strava
df = df_univ_nd
fig = px.scatter(df, x=chsen_ftr_strava, y="people_counter_data",\
                  color="site",trendline="ols")
fig.show()

results = px.get_trendline_results(fig)

# Store univariate regression outputs
site_values=results['site'].values

str_regrn_nd=[]
for sites in site_values:
    df_site=df[df['site']==sites]
    scaler = StandardScaler()
    df_site[["people_counter_data",chsen_ftr_strava]]=scaler.fit_transform(df_site[["people_counter_data",chsen_ftr_strava]])
    lm = pg.linear_regression(df_site[['people_counter_data']],df_site[chsen_ftr_strava])
    lm['site']=sites
    str_regrn_nd.append(lm)
    
str_regrn_nd=pd.concat(str_regrn_nd).reset_index(drop=True)

str_regrn_nd[['r2','site']].drop_duplicates().set_index('site').sort_values(by='r2',ascending=False).\
plot(kind='barh',figsize=(8,6))

In [ ]:
str_regrn_ne_nd=pd.concat([str_regrn_ne,str_regrn_nd])

str_regrn_ne_nd=str_regrn_ne_nd[str_regrn_ne_nd['names'].isin(['people_counter_data'])]

str_regrn_ne_nd=str_regrn_ne_nd[['names','coef','pval','site']]

str_regrn_ne_nd=str_regrn_ne_nd[str_regrn_ne_nd['pval']<0.05]

str_regrn_ne_nd_countr=str_regrn_ne_nd[str_regrn_ne_nd['names']=='people_counter_data']


str_regrn_ne_nd_countr[['coef','site']].drop_duplicates().set_index('site').sort_values(by='coef',ascending=False).\
plot(kind='barh',figsize=(8,6))

In [ ]:
# River and Canal Trust: No counter data available

df_univ_cr=df_count_cr_strava[['Date','site',chsen_ftr_strava]].copy()


df_univ_cr=df_univ_cr.set_index('Date').sort_index()


df_univ_cr=df_univ_cr[['site',chsen_ftr_strava]]

## Merge static and dynamic features

<a id='dest_6'></a>

### 1. Merge OSM data with counter data

In [ ]:
# Merge dynamic features with static (spatial) features: Natural England

df_univ_ne=df_univ_ne.reset_index().merge(df_pois_all_sites,on=['site'],how='inner').\
set_index('Date').sort_index()

df_univ_ne=df_univ_ne.reset_index()

df_univ_ne=df_univ_ne.sort_values(['Date', 'site'], ascending=[True, True])

In [ ]:
# Merge dynamic features with static (spatial) features: North Downs

df_univ_nd=df_univ_nd.reset_index().merge(df_pois_all_sites,on=['site'],how='inner').\
set_index('Date').sort_index()

df_univ_nd=df_univ_nd.reset_index()

df_univ_nd=df_univ_nd.sort_values(['Date', 'site'], ascending=[True, True])

In [ ]:
# Merge dynamic features with static (spatial) features: Canal and River Trust

df_univ_cr=df_univ_cr.reset_index().merge(df_pois_all_sites,on=['site'],how='inner').\
set_index('Date').sort_index()

df_univ_cr=df_univ_cr.reset_index()

df_univ_cr=df_univ_cr.sort_values(['Date', 'site'],ascending=[True, True])

### 2. Merge Weather data with counter data

Filter weather data for people counter sites

https://meteostat.net/en/blog/obtain-weather-data-any-location-python

In [ ]:
weather_df=pd.read_pickle(data_folder+'weather_df.pkl')

weather_df['Date']=weather_df['Date'].astype(str)

In [ ]:
df_univ_ne['Date']=df_univ_ne['Date'].astype(str)

df_univ_ne=df_univ_ne.copy().merge(weather_df,on=['Date','site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_ne.isna().sum(axis=0).sum(),df_univ_ne['site'].unique().shape[0]))

In [ ]:
df_univ_nd['Date']=df_univ_nd['Date'].astype(str)

df_univ_nd=df_univ_nd.copy().merge(weather_df,on=['Date','site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_nd.isna().sum(axis=0).sum(),df_univ_nd['site'].unique().shape[0]))

In [ ]:
df_univ_cr['Date']=df_univ_cr['Date'].astype(str)

df_univ_cr=df_univ_cr.copy().merge(weather_df,on=['Date','site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_cr.isna().sum(axis=0).sum(),df_univ_cr['site'].unique().shape[0]))

### 3. Merge Demographic data with counter data

In [ ]:
df_demg_ne.rename(columns={'counter':'site'},inplace=True)

df_demg_nd.rename(columns={'counter':'site'},inplace=True)

df_demg_cr.rename(columns={'counter':'site'},inplace=True)

In [ ]:
df_univ_ne=df_univ_ne.merge(df_demg_ne,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_ne.isna().sum(axis=0).sum(),df_univ_ne['site'].unique().shape[0]))

In [ ]:
df_univ_nd=df_univ_nd.merge(df_demg_nd,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_nd.isna().sum(axis=0).sum(),df_univ_nd['site'].unique().shape[0]))

In [ ]:
df_univ_cr=df_univ_cr.merge(df_demg_cr,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_cr.isna().sum(axis=0).sum(),df_univ_cr['site'].unique().shape[0]))

### 4. Merge green infrastructure data with counter data



In [ ]:
# Select the required variables

df_grn_inf=pd.read_pickle(data_folder+'green_blue_prow_ftrs.pkl')

df_grn_inf.rename(columns={'counter':'site'},inplace=True)


sel_green_ftrs=['site','accessible_green_space_area','waterside_length_km','PROW_Total_length_km']

df_grn_inf=df_grn_inf[sel_green_ftrs]



df_grn_inf=df_grn_inf.set_index('site')



green_infrastrcur_ftrs=list(df_grn_inf.columns)

df_grn_inf=df_grn_inf.reset_index()

#df_grn_inf=df_grn_inf.merge(area_sites_oa.rename(columns={'counter':'site'}),on=['site'],how='inner')

#df_grn_inf['accessible_green_space_area']=df_grn_inf['accessible_green_space_area'].div(df_grn_inf['area_sq_km'])

#df_grn_inf['waterside_length_km']=df_grn_inf['waterside_length_km'].div(np.sqrt(df_grn_inf['area_sq_km']))

#df_grn_inf['PROW_Total_length_km']=df_grn_inf['PROW_Total_length_km'].div(np.sqrt(df_grn_inf['area_sq_km']))

In [ ]:
# MERGE GREEN INFRASTRUCTURE FEATURES on counter data: Natural England 

df_univ_ne=df_univ_ne.merge(df_grn_inf,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_ne.isna().sum(axis=0).sum(),df_univ_ne['site'].unique().shape[0]))

In [ ]:
# MERGE GREEN INFRASTRUCTURE FEATURES on counter data: North Downs Way 


df_univ_nd=df_univ_nd.merge(df_grn_inf,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_nd.isna().sum(axis=0).sum(),df_univ_nd['site'].unique().shape[0]))

In [ ]:
# MERGE GREEN INFRASTRUCTURE FEATURES on counter data: canal and river trust 


df_univ_cr=df_univ_cr.merge(df_grn_inf,on=['site'],how='inner')

print('Number of null values {} and number of unique counter sites is {}'.\
      format(df_univ_cr.isna().sum(axis=0).sum(),df_univ_cr['site'].unique().shape[0]))

### 5. Merge season information with counter data

In [ ]:
# Introduce seasons
        
df_univ_ne['season']=df_univ_ne['Date'].apply(lambda x: x.split('-')[1]).apply(lambda x : get_season(x))


df_univ_nd['season']=df_univ_nd['Date'].apply(lambda x: x.split('-')[1]).apply(lambda x : get_season(x))


df_univ_cr['season']=df_univ_cr['Date'].apply(lambda x: x.split('-')[1]).apply(lambda x : get_season(x))


df_univ_ne=pd.get_dummies(df_univ_ne,columns=['season'])

df_univ_nd=pd.get_dummies(df_univ_nd,columns=['season'])

df_univ_cr=pd.get_dummies(df_univ_cr,columns=['season'])

### 6. Merge land habitat and land type information with counter data

In [ ]:
# Read the data: each site is classified into habitat type and land type class

land_habitat_df=pd.read_pickle(habitat_cluster_file)

del land_habitat_df['geometry']

land_habitat_df.rename(columns={'counter':'site'},inplace=True)

land_habitat_df.rename(columns={'labels':'habitat_type_labels'},inplace=True)



land_cluster_df=pd.read_pickle(land_cluster_file)

del land_cluster_df['geometry']

land_cluster_df.rename(columns={'counter':'site'},inplace=True)

land_cluster_df.rename(columns={'labels':'land_type_labels'},inplace=True)

print(list(land_cluster_df['land_type_labels'].unique()))

print(list(land_habitat_df['habitat_type_labels'].unique()))



df_univ_ne=df_univ_ne.merge(land_cluster_df[['site','land_type_labels']],on=['site'],how='inner')
df_univ_ne=df_univ_ne.merge(land_habitat_df[['site','habitat_type_labels']],on=['site'],how='inner')

df_univ_nd=df_univ_nd.merge(land_cluster_df[['site','land_type_labels']],on=['site'],how='inner')
df_univ_nd=df_univ_nd.merge(land_habitat_df[['site','habitat_type_labels']],on=['site'],how='inner')

df_univ_cr=df_univ_cr.merge(land_cluster_df[['site','land_type_labels']],on=['site'],how='inner')
df_univ_cr=df_univ_cr.merge(land_habitat_df[['site','habitat_type_labels']],on=['site'],how='inner')

### 7. Merge Dog ownership data with counter data

In [ ]:
# read the data
df_dog_ownrshp=pd.read_pickle(dog_ownership)

df_dog_ownrshp.rename(columns={'counter':'site'},inplace=True)

df_univ_ne=df_univ_ne.merge(df_dog_ownrshp,on=['site'],how='left').fillna(0)

df_univ_nd=df_univ_nd.merge(df_dog_ownrshp,on=['site'],how='left').fillna(0)

df_univ_cr=df_univ_cr.merge(df_dog_ownrshp,on=['site'],how='left').fillna(0)

In [ ]:
#demog_ftrs=[x for x in list(all_sites_demographic.select_dtypes(include=[np.number]).columns.values)]

#print(demog_ftrs)

ftrs_to_keep=list(set(df_univ_ne.columns).intersection(df_univ_nd.columns))

ftrs_to_keep=list(set(ftrs_to_keep).intersection(df_univ_cr.columns))

print(sorted(ftrs_to_keep))

In [ ]:
# Get all the data for all the sites
df_sites_ne_nd_cr=pd.concat([df_univ_ne[ftrs_to_keep+[target]],df_univ_nd[ftrs_to_keep+[target]],\
                             df_univ_cr[ftrs_to_keep]],axis=0).reset_index(drop=True)

df_sites_ne_nd_cr.to_pickle(data_folder+'complete_dataset.pkl')

In [ ]:
df_sites_ne_nd_cr.dropna().groupby('Date')['site'].count().plot(style='-o')
plt.ylabel('Number of monitoring sites with training data available')